In [9]:
import xgboost as xgb

In [1]:
import sys, xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\Mustapha\Desktop\ML-Project-Scratch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load feature engineered datasets
train_df = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\processed\\feature_engineered_train.csv')
eval_df = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\processed\\feature_engineered_eval.csv')

# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [3]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [7]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
mlflow.set_tracking_uri("/Users/Mustapha/Desktop/ML-Project-Scratch/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2026/01/23 17:44:46 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.


2026/01/23 17:44:46 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.


[I 2026-01-23 17:44:46,570] A new study created in memory with name: no-name-d9e29234-4cf7-4fef-afdf-230bdf6ab286
[I 2026-01-23 17:45:02,653] Trial 0 finished with value: 76526.7123044742 and parameters: {'n_estimators': 273, 'max_depth': 8, 'learning_rate': 0.12163251556074833, 'subsample': 0.5520752823216983, 'colsample_bytree': 0.5961183407823035, 'min_child_weight': 10, 'gamma': 0.958796366022549, 'reg_alpha': 8.338150604385063e-05, 'reg_lambda': 0.0002877165200756515}. Best is trial 0 with value: 76526.7123044742.
[I 2026-01-23 17:45:20,184] Trial 1 finished with value: 79018.88583140651 and parameters: {'n_estimators': 251, 'max_depth': 8, 'learning_rate': 0.012083575517234092, 'subsample': 0.6151956450441158, 'colsample_bytree': 0.9619101943914852, 'min_child_weight': 5, 'gamma': 3.5436835935069286, 'reg_alpha': 1.7330921527668205e-06, 'reg_lambda': 2.1897610935956465e-08}. Best is trial 0 with value: 76526.7123044742.
[I 2026-01-23 17:45:31,553] Trial 2 finished with value: 903

2026/01/23 17:44:46 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.


[I 2026-01-23 17:44:46,570] A new study created in memory with name: no-name-d9e29234-4cf7-4fef-afdf-230bdf6ab286
[I 2026-01-23 17:45:02,653] Trial 0 finished with value: 76526.7123044742 and parameters: {'n_estimators': 273, 'max_depth': 8, 'learning_rate': 0.12163251556074833, 'subsample': 0.5520752823216983, 'colsample_bytree': 0.5961183407823035, 'min_child_weight': 10, 'gamma': 0.958796366022549, 'reg_alpha': 8.338150604385063e-05, 'reg_lambda': 0.0002877165200756515}. Best is trial 0 with value: 76526.7123044742.
[I 2026-01-23 17:45:20,184] Trial 1 finished with value: 79018.88583140651 and parameters: {'n_estimators': 251, 'max_depth': 8, 'learning_rate': 0.012083575517234092, 'subsample': 0.6151956450441158, 'colsample_bytree': 0.9619101943914852, 'min_child_weight': 5, 'gamma': 3.5436835935069286, 'reg_alpha': 1.7330921527668205e-06, 'reg_lambda': 2.1897610935956465e-08}. Best is trial 0 with value: 76526.7123044742.
[I 2026-01-23 17:45:31,553] Trial 2 finished with value: 903

Best params: {'n_estimators': 732, 'max_depth': 9, 'learning_rate': 0.03486028826847422, 'subsample': 0.7455099729388772, 'colsample_bytree': 0.7232194907713484, 'min_child_weight': 8, 'gamma': 1.2662078585274723, 'reg_alpha': 0.019064382043892558, 'reg_lambda': 0.44887936362751646}


In [11]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.sklearn.log_model(best_model, artifact_path="model")

2026/01/23 18:03:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Final tuned model performance:
MAE: 30621.54376821812
RMSE: 71175.69003882815
R²: 0.9608507538567431


2026/01/23 18:03:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
